# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [15]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [16]:
context = [ {'role':'system', 'content':"""
    Read these three tables:
             
    Table 1: visitors
    CREATE TABLE visitors (
        visitor_id INT PRIMARY KEY,
        name VARCHAR(100),
        order_status VARCHAR(100)
    )
             
    Example Data:
        | visitor_id | name               | order_status    |
        |------------|--------------------|-----------------|
        | 1          | Clemant            | completed       |
        | 2          | Svengana           | pending        
             
    Table 2: orders
    CREATE TABLE orders (
        order_id INT PRIMARY KEY,
        order_date DATE,
        FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id)
    )
    
    Example Data:
        | order_id | order_date | visitor_id |
        |----------|------------|------------|
        | 101      | 2024-12-01 | 1          |
        | 102      | 2024-12-02 | 2          |         

    Table 3: prices
    CREATE TABLE prices (
        price_id INT PRIMARY KEY,
        price_amount DECIMAL,
        FOREIGN KEY (order_id) REFERENCES orders(order_id)
    )
             
    Example Data:
        | price_id | price_amount | order_id |
        |----------|--------------|----------|
        | 201      | 19.99        | 101      |
        | 202      | 29.99        | 102      |

"""} ]



In [17]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Your job is to respond to each question that a user sends about the data in the database \ 
with the SQLite command that could retrieve this information. If the question cannot be answered \
with a SQL command, inform the user ferociously that only SQL topics are alloqwed. Maintain the \
SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions \
for the table provided above.
                 
Refer to the tables defined earlier to construct your answers.

Question: What is the highest ticket price for our festival?
Response:
SELECT MAX(price_amount)
FROM prices;

Question: On which date was the first order placed?
Response:
SELECT order_date
FROM orders
ORDER BY order_date ASC
LIMIT 1;
                         
"""
})

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/var/folders/t4/gbgxnn9j3sj48qv4ydr2n_100000gn/T/ipykernel_99756/2525243025.py:2: SyntaxWarning: invalid escape sequence '\ '
  context.append( {'role':'system', 'content':"""


In [18]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [19]:
#new
context_user = context.copy()
print(return_CCRMSQL("How many visitors have completed their orders?", context_user))

Response:
SELECT COUNT(*)
FROM visitors
WHERE order_status = 'completed';


In [20]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("How many visitors have completed their orders?", old_context_user))

This is your SQL:
```sql
SELECT COUNT(*) AS completed_orders
FROM orders
WHERE status = 'completed';
```
This SQL query counts the number of orders that have been completed by selecting the count of rows where the status column is marked as 'completed' in the orders table.


In [21]:
#new
print(return_CCRMSQL("Give me all names that start with a 'z'", context_user))

I'm sorry, but the request to retrieve names starting with a specific letter like 'z' is not directly possible with the provided tables and data.


In [23]:
#old
print(return_CCRMSQL("Give me all names that start with a 'z'", old_context_user))

This is your SQL:
```sql
SELECT name FROM employees WHERE name LIKE 'z%';
```

This SQL query selects all names from the employees table where the name starts with the letter 'z'. The '%' is a wildcard character that matches any sequence of characters.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

Findings:
- new context with samples ensures that output of model sticks exactly to defined output
- for some reason, the response based on my own context to the question "Give me all names that start with a 'z'" said that this is not possible, while the old context resulted in a correct answer